In [ ]:
import pandas as pd

In [159]:
# Variables
conventionals = {
    'Erdgas [MWh] Originalauflösungen': 'fossile_gas',
    'Braunkohle [MWh] Originalauflösungen': 'lignite',
    'Steinkohle [MWh] Originalauflösungen': 'hard_coal',
    'Sonstige Konventionelle [MWh] Originalauflösungen': 'other_conventionals'
}

In [173]:
# Preprocess Generation Data
## Read in generation data for each region
areas_df_dict = {
    'f_hertz': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_50Hertz_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'amprion': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_Amprion_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'tennet': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_TenneT_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'transnet_bw': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_TransnetBW_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    )
}

## Convert regional generation data
for area in areas_df_dict:
    # Rename conventional columns
    for conv in conventionals:
        if conv in areas_df_dict[area].columns:
            areas_df_dict[area] = areas_df_dict[area].rename(columns={conv: conventionals[conv]})

    # Rename date columns
    areas_df_dict[area] = areas_df_dict[area].rename(columns={
        'Datum von': 'date_from',
        'Datum bis': 'date_to'
    })

    # Drop all columns that are not in conventionals (plus date columns)
    keep_cols = {'date_from', 'date_to', *conventionals.values()}
    areas_df_dict[area] = areas_df_dict[area].loc[:, [c for c in areas_df_dict[area].columns if c in keep_cols]]

    # Transform date columns in datetime
    areas_df_dict[area]['date_from'] = pd.to_datetime(
        areas_df_dict[area]['date_from'],
        format="%d.%m.%Y %H:%M"
    )
    areas_df_dict[area]['date_to'] = pd.to_datetime(
        areas_df_dict[area]['date_to'],
        format="%d.%m.%Y %H:%M"
    )

    # Transform columns to numeric
    for col in areas_df_dict[area].columns:
        if col not in ("date_from", "date_to"):
            if not pd.api.types.is_float_dtype(areas_df_dict[area][col]):
                areas_df_dict[area][col] = (
                    areas_df_dict[area][col]
                    .astype(str)
                    .str.replace(".", "", regex=False)
                    .str.replace(',', '.', regex=False)
                )
                areas_df_dict[area][col] = pd.to_numeric(areas_df_dict[area][col], errors='coerce')

    # Aggregate conventional generation
    areas_df_dict[area]['total_generation'] = areas_df_dict[area].sum(axis=1, numeric_only=True)

In [174]:
print(areas_df_dict['f_hertz'].columns)
print(areas_df_dict['f_hertz'].dtypes)

Index(['date_from', 'date_to', 'lignite', 'hard_coal', 'fossile_gas',
       'other_conventionals', 'total_generation'],
      dtype='object')
date_from              datetime64[ns]
date_to                datetime64[ns]
lignite                       float64
hard_coal                     float64
fossile_gas                   float64
other_conventionals           float64
total_generation              float64
dtype: object


In [178]:
areas_df_dict['f_hertz'].head()

,date_from,date_to,lignite,hard_coal,fossile_gas,other_conventionals,total_generation
0,2023-01-01 00:00:00,2023-01-01 00:15:00,604.50,112.75,112.25,178.50,1008.00
1,2023-01-01 00:15:00,2023-01-01 00:30:00,605.75,111.75,112.25,178.25,1008.00
2,2023-01-01 00:30:00,2023-01-01 00:45:00,609.75,110.75,112.50,179.50,1012.50
3,2023-01-01 00:45:00,2023-01-01 01:00:00,609.00,110.00,112.25,177.25,1008.50
4,2023-01-01 01:00:00,2023-01-01 01:15:00,606.75,109.75,112.50,177.75,1006.75


In [175]:
# Preprocess Emissions Data
## Read in emissions data for each year and merge to one dataframe
emissions_2023 = pd.read_csv('../data/raw/agora/emissions_ger_2023_hourly.csv')
emissions_2024 = pd.read_csv('../data/raw/agora/emissions_ger_2024_hourly.csv')

## Concatenate and drop duplicate (last 2023 and first 2024)
emissions = pd.concat([emissions_2023, emissions_2024])
emissions.drop_duplicates(keep = 'first', inplace=True)

## Rename columns
emissions = emissions.rename(columns={
    'Braunkohle': 'lignite',
    'Erdgas': 'fossile_gas',
    'Andere': 'other_conventionals',
    'Steinkohle': 'hard_coal',
    'Absolute Emissionen': 'total_emissions',
    'CO₂-Emissionsfaktor des Strommix': 'aef'
})

## Convert types
emissions['Datetime'] = pd.to_datetime(emissions['Datetime'], format='%Y-%m-%dT%H:%M:%S')
emissions = emissions.set_index('Datetime')

In [177]:
emissions.index

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 01:00:00',
               '2023-01-01 02:00:00', '2023-01-01 03:00:00',
               '2023-01-01 04:00:00', '2023-01-01 05:00:00',
               '2023-01-01 06:00:00', '2023-01-01 07:00:00',
               '2023-01-01 08:00:00', '2023-01-01 09:00:00',
               ...
               '2024-12-31 15:00:00', '2024-12-31 16:00:00',
               '2024-12-31 17:00:00', '2024-12-31 18:00:00',
               '2024-12-31 19:00:00', '2024-12-31 20:00:00',
               '2024-12-31 21:00:00', '2024-12-31 22:00:00',
               '2024-12-31 23:00:00', '2025-01-01 00:00:00'],
              dtype='datetime64[ns]', name='Datetime', length=17543, freq=None)

In [176]:
# Save dataframes to a file

for area in areas_df_dict:
    min = areas_df_dict[area]['date_from'].min()
    areas_df_dict[area].to_csv(f'../data/interim/generation_{area}_.csv')

Index(['other_conventionals', 'lignite', 'hard_coal', 'fossile_gas',
       'total_emissions', 'aef'],
      dtype='object')